*Автор: Татьяна Рогович*

# Анализ данных в Python

## Введение в ML. Задача регрессии

Этот блокнот составлен на основе большого блокнота, который доступен по этой [ссылке](https://www.kaggle.com/dansbecker/your-first-machine-learning-model). В нем присутствуют дополнительные комментарии и другие примеры работы алгоритма. Если хотите как следует разобраться с деревьями на базовом уровне - обязательно читайте.

### Загружаем наши данные

Нашим датасетом будет информация о домах, которая включает в себя множество параметров, например, количество и размер этажей дома, год постройки, общая площадь, цена и многое другое (остальные столбцы, можете изучить самостоятельно).

In [1]:
import pandas as pd

#Загружаем данные
home_data = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/house_data.csv')

### Выбор целевой переменной

В нашем случае, все более-менее очевидно. Это цена на дом. Обычно, цель предсказания помечают y.

In [2]:
y = home_data.SalePrice

### Выбор признаков (Features)

Столбцы, которые есть в нашей модели и которые в последствии будут использованы для предсказания, называются признаками (features). В нашем случае, эти колонки будут определять стоимость дома. Иногда используются все колонки, кроме той на которую делается предсказание. А вдругих случаях, лучше выбрать только часть из них. Как определиться? Для этого проводят разведывательный анализ или сравнивают качество работы моделей на разных выборках признаков.

Мы будет использовать не все столбцы, чтобы алгоритм все считал быстрее. Добавим в модель только те столбцы, которые сильнее всего коррелировали с ценой, когда мы исследовали данные.

In [3]:
feature_columns = ['OverallQual', 'GrLivArea', 'GarageArea', 'TotalBsmtSF', 'FullBath', 'YearBuilt', 'YearRemodAdd']

Обычно, такие данные обозначаются X:

In [4]:
X = home_data[feature_columns]

Чтобы обучить модель, нам надо разбить нашу выборку на тестовую и тренировочную.

In [5]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3)

Сколько элементов попало в подвыборки?

In [6]:
print(X_tr.shape, X_te.shape)

(1022, 7) (438, 7)


### Строим модель

Мы будем использовать модуль scikit-learn для создания модели. Scikit-learn одна из самых популярных библиотек для моделирования данных, хранящихся в датафреймах. 

Для построения модели нужно выполнить следующие шаги: <br>

**Define**: Какого типа будет модель? Линейная регрессия? Какой-то другой тип?<br>
**Fit**: запускаем модель на тренировочных данных (обучаем ее), чтобы алгоритм нашел в них некие закономерности и зависимости.<br>
**Predict**: Предсказать результат.<br>
**Evaluate**: Определить насколько точным оказалось предсказывание, оценить качество модели<br>


### Обучение линейной модели

Мы готовы к тому, чтобы построить модель машинного обучения. Обучим линейную модель на наших данных. Для этого импортируем LinearRegression из модуля sklearn:

In [7]:
from sklearn.linear_model import LinearRegression

Обучение модели в sklearn всегда состоит из двух шагов - создания модели и вызова функции fit:

In [8]:
model = LinearRegression()
model.fit(X_tr, y_tr)

LinearRegression()

После обучения "внутри" модели появились найденные веса:

In [9]:
model.coef_

array([19953.40641092,    52.86601583,    47.03909824,    22.07748441,
       -2156.45339994,   250.61436687,   297.70178945])

### Предсказания линейной модели

Выполним предсказания на двух выборках, обучающей и тестовой, и сохраним их в переменные preds_tr и preds_te. Для этого воспользуемся функцией predict:

In [10]:
preds_tr = model.predict(X_tr)
preds_te = model.predict(X_te)

Посмотрим на предсказания на первых 10 объектах:

In [11]:
preds_te[:10]

array([124884.53343939, 103154.86982183, 200231.70828836, 372849.32615521,
       218145.3292307 , 179534.56028436, 299008.95511057, 214439.52222616,
       193867.56279722, 298267.08557509])

А вот целевые значения из выборки:

In [12]:
y_te[:10].values

array([ 89500, 129500, 184000, 485000, 194500, 153500, 385000, 205000,
       163500, 350000])

### Оценивание качества

Сравнивать предсказания и правильные ответы вручную удобно только на маленькой выборке данных. Для оценивания качества работы модели лучше измерить метрику качества. Для этого в sklearn есть подмодуль metrics. 

Воспульзуемся метрикой MEA (Mean Absolute Error) - Cредняя Aбсолютная Ошибка. Ее можно представить в таком виде: ошибка = реальная цена − предсказанная цена. Например, если цена дома 150 000, мы предсказали цены в 100 000, то ошибка будет 50 000. Для ошибок в отрицательную сторону берем модуль. Это одна из простейших метрик.

Импортируем функцию mean_absolute_error.

In [13]:
from sklearn.metrics import mean_absolute_error 

Вычисляем ошибку на обучающей выборке:

In [14]:
mean_absolute_error(y_tr, preds_tr)

23677.879140493154

Вычисляем ошибку на тестовой выборке:

In [15]:
mean_absolute_error(y_te, preds_te)

24380.977270085947

### Анализ модели

Посмотрим на веса модели. Запишем их в таблицу со столбцами "название признака" и "вес признака" и отсортируем по значениям весов:

In [16]:
weights_data = {"веса":model.coef_, "признаки": X.columns}
weights = pd.DataFrame(weights_data)
weights.sort_values("веса")

,веса,признаки
4,-2156.453400,FullBath
3,22.077484,TotalBsmtSF
2,47.039098,GarageArea
1,52.866016,GrLivArea
5,250.614367,YearBuilt
6,297.701789,YearRemodAdd
0,19953.406411,OverallQual


Логично ли распределились вклады признаков?

### Задания для самостоятельной работы

Перед выполнением заданий обязательно выполните все ячейки выше. Для этого мжно нажать на эту ячейку, далее в меню Cell выбрать Run all above.

При выполнении заданий используйте примеры кода, данные выше, а также [__таблицу с подсказками__](https://github.com/nadiinchi/intro_sklearn/blob/master/Cheatsheet.pdf).

Мы обучили линейную модель на данных недвижимости Бостона. Теперь давайте рассмотрим еще две модели - метод k ближайших соседей (kNN) и нейронную сеть.

#### Задача 1. Обучение kNN

Обучите метод k ближайших соседей на данных. Метод уже импортирован в следующей ячейке.

In [17]:
from sklearn.neighbors import KNeighborsRegressor

In [1]:
# ваш код здесь

#### Задача 2. Предсказания kNN

Выполните предсказания с помощью метода k ближайших соседей на обучающей и тестовой выборках.

In [2]:
# ваш код здесь

#### Задача 3. Качество kNN

Выведите значение ошибки метода k ближайших соседей на обучающей и тестовой выборках.

In [3]:
# ваш код здесь

#### Задача 4. Улучшаем kNN

У метода k ближайших соседей есть важный гиперпараметр - число соседей k. В sklearn он обозначен n_neighbors и задается следующим образом:

`n_neighbors = 1`

In [4]:
# ваш код здесь

Попробуйте использовать n_neighbors, равное 1, 3, 10, 100. Для каждого значения повторите шаги предыдущих трех задач: обучите модель, выполните предсказания, выведите ошибку на обучающей и тестовой выборке. Выберите n_neighbors с наименьшей ошибкой на тестовой выборке. Используйте цикл по четырем значениям n_neighbors.

In [5]:
# ваш код здесь